https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

https://medium.com/python-data/efficient-frontier-portfolio-optimization-with-python-part-2-2-2fe23413ad94

https://anaconda.org/mcg/portfolio/notebook

https://towardsdatascience.com/efficient-frontier-portfolio-optimisation-in-python-e7844051e7f



In [1]:
import pandas as pd
import numpy as np
import datetime
import time
import json
import random
import cvxpy as cvx
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from operator import itemgetter

plotly.tools.set_credentials_file(username='Shenghao1993', api_key='EsC2v5QMJhXCNQHwX5gY')

# Acquire ETF price data

In [2]:
etf_prices = pd.read_csv('data/prices.txt', sep=',')
etf_prices = etf_prices.rename(columns={etf_prices.columns[0]: "date"}).set_index('date')
#etf_prices = etf_prices.sample(n=20, random_state=1, axis=1)
#etf_prices = etf_prices[:400]
etf_prices = etf_prices.iloc[:, :-1]
etf_prices.head()

,AGG,DBC,DFE,DIA,DXJ,EEM,EFA,EWG,EWH,EWI,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP
date,,,,,,,,,,,,,,,,,,,,,
2007-01-02,99.91,23.80,65.66,124.47,55.28,38.34,73.51,27.15,16.16,33.42,...,56.58,36.92,35.23,23.33,26.29,36.82,33.50,38.52,47.66,36.64
2007-01-03,99.91,23.80,65.66,124.47,55.28,38.34,73.51,27.15,16.16,33.42,...,56.58,36.92,35.23,23.33,26.29,36.82,33.50,38.52,47.66,36.64
2007-01-04,100.12,23.18,65.60,124.68,55.95,37.81,73.32,27.06,15.99,33.34,...,55.55,36.80,35.08,23.69,26.33,36.77,33.88,38.51,47.42,36.16
2007-01-05,100.05,23.20,65.02,123.74,54.78,36.71,72.20,26.61,15.87,32.91,...,55.92,36.49,34.96,23.50,26.14,36.11,33.69,38.25,47.01,36.50
2007-01-08,100.11,23.22,65.26,124.14,54.86,36.97,72.30,26.53,16.27,32.95,...,55.92,36.67,35.09,23.57,26.18,36.09,33.73,38.31,47.49,36.66


In [43]:
price_data_for_plot = []
for col in etf_prices.columns:
    trace = go.Scatter(
                x=etf_prices.index,
                y=etf_prices[col],
                name=col,
                opacity = 0.8)
    price_data_for_plot.append(trace)

layout = dict(
    title = "ETF Prices from 2007 to 2016",
    xaxis = dict(
        range = [etf_prices.index])
)

fig = dict(data=price_data_for_plot, layout=layout)
py.iplot(fig, filename = "ETF Prices from 2007 to 2016")

The draw time for this plot will be slow for all clients.


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [3]:
etf_returns_df = np.log(etf_prices) - np.log(etf_prices.shift(1))
etf_returns_df = etf_returns_df.dropna()
etf_returns_df.head()

,AGG,DBC,DFE,DIA,DXJ,EEM,EFA,EWG,EWH,EWI,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP
date,,,,,,,,,,,,,,,,,,,,,
2007-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2007-01-04,0.002100,-0.026396,-0.000914,0.001686,0.012047,-0.013920,-0.002588,-0.003320,-0.010576,-0.002397,...,-0.018372,-0.003256,-0.004267,0.015313,0.001520,-0.001359,0.011279,-0.000260,-0.005048,-0.013187
2007-01-05,-0.000699,0.000862,-0.008881,-0.007568,-0.021133,-0.029524,-0.015393,-0.016770,-0.007533,-0.012981,...,0.006639,-0.008460,-0.003427,-0.008053,-0.007242,-0.018112,-0.005624,-0.006774,-0.008684,0.009359
2007-01-08,0.000600,0.000862,0.003684,0.003227,0.001459,0.007058,0.001384,-0.003011,0.024892,0.001215,...,0.000000,0.004921,0.003712,0.002974,0.001529,-0.000554,0.001187,0.001567,0.010159,0.004374
2007-01-09,0.000000,-0.008651,-0.000153,-0.000564,0.007084,-0.022430,0.000968,0.004513,-0.002462,-0.000607,...,-0.008621,-0.001638,-0.000570,0.001272,0.002289,0.000831,-0.001187,0.005467,-0.011436,-0.010420


We calculate the return using

$$ R = p^T w $$

where $R$ is the expected return, $p^T$ is the transpose of the vector for the mean
returns for each time series and w is the weight vector of the portfolio. $p$ is a $N \times 1$
column vector, so $p^T$ turns is a $1 \times N$ row vector which can be multiplied with the
$N \times 1$ weight (column) vector w to give a scalar result. This is equivalent to the dot
product used in the code. Keep in mind that `Python` has a reversed definition of
rows and columns and the accurate `NumPy` version of the previous equation would
be `R = w * p.T`

Next, we calculate the standard deviation

$$\sigma = \sqrt{w^T C w}$$

where $C$ is the $N \times N$ covariance matrix of the returns. Please
note that if we simply calculated the simple standard deviation with the appropriate weighting using `std(array(ret_vec).T*w)` we would get a slightly different
’bullet’. This is because the simple standard deviation calculation would not take
covariances into account. In the covariance matrix, the values on the diagonal
represent the simple variances of each asset, while the off-diagonal entries are the variances between the assets. By using ordinary `std()` we effectively only regard the
diagonal and miss the rest. A small but significant difference.

# Explore return of the first year (2007)

In [5]:
first_year_returns = etf_returns_df[:252]
first_year_returns_stats = []
for col in first_year_returns.columns:
    mean_variance = {}
    mean_variance['asset'] = col
    mean_variance['mean'] = first_year_returns[col].mean()
    mean_variance['variance'] = first_year_returns[col].std()
    first_year_returns_stats.append(mean_variance)

first_year_returns_stats_df = pd.DataFrame(first_year_returns_stats)
first_year_returns_stats_df = first_year_returns_stats_df.set_index('asset')
first_year_returns_stats_df.shape

(52, 2)

In [7]:
return_data = [
    go.Scatter(
        x=first_year_returns_stats_df['variance'],
        y=first_year_returns_stats_df['mean'],
        mode='markers+text',
        text=first_year_returns_stats_df.index,
        textposition='bottom center'
    )
]

layout = go.Layout(
    title='Mean and standard deviation of daily return in 2017',
    xaxis= {'title': 'Volatility (standard deviation) of daily return'},
    yaxis= {'title': "Mean daily return"}
)

fig = go.Figure(data=return_data, layout=layout)
py.iplot(fig, filename='mean-variance-return')

# Portfolio Optimization

Assumptions:
1. Uniform annual expense ratio of 0.75%.
2. Frequency of portfolio rebalance: once a month (25 trading days).
3. Expected annual return: 10%.

https://www.investopedia.com/ask/answers/071816/how-are-etf-fees-deducted.asp

In [9]:
def get_covariance_returns(returns):
    """
    Calculate covariance matrices.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    returns_covariance  : 2 dimensional Ndarray
        The covariance of the returns
    """
    
    return np.cov(returns.fillna(0).T)

In [10]:
def adjust_weights(weights):
    adjusted_weights = weights.copy()
    assets = list(weights.keys())
    fracs = list(weights.values())
    fracs = fracs[:-1]
    fracs = [round(frac, 4) for frac in fracs]

    min_weight_asset_id = min(enumerate(fracs), key=itemgetter(1))[0]
    min_weight_asset = assets[min_weight_asset_id]
    adjusted_weights[min_weight_asset] = 1 - sum(fracs) + adjusted_weights[min_weight_asset]
    return adjusted_weights

In [16]:
def get_optimal_weights(covariance_returns, return_vec, expected_annual_return):
    """
    Find the optimal weights.

    Parameters
    ----------
    covariance_returns : 2 dimensional Ndarray
        The covariance of the returns
    return_vec : 2 dimensional Ndarray
        Index weights for all tickers at a period in time
    expected_annual_return : float
        Expected rate of annual portfolio return
        
    Returns
    -------
    reduced_weights : 1 dimensional Ndarray
        The solution for x
    etfs_selection: 1 dimensional Ndarray
    
    """
    M = 100
    num_assets = covariance_returns.shape[1]
    initial_weights = cvx.Variable(num_assets)
    reduced_weights = cvx.Variable(num_assets)
    etfs_selection = cvx.Variable(num_assets, boolean=True)
    portfolio_variance = cvx.quad_form(reduced_weights, covariance_returns)
    
    ## Objective function
    objective = cvx.Minimize(portfolio_variance)
    
    ## Constraints
    mu = return_vec * reduced_weights
    rate_of_return = np.log(1 + expected_annual_return) * len(return_vec) / 252
    constraints = [initial_weights >= 0.001, initial_weights <= 0.3, sum(reduced_weights)==1,
                   reduced_weights - initial_weights >= -M * (1 - etfs_selection),
                   reduced_weights - initial_weights <= M * (1 - etfs_selection),
                   reduced_weights >= -M * etfs_selection, reduced_weights <= M * etfs_selection,
                   sum(etfs_selection) >= 5, sum(etfs_selection) <= 10, sum(mu) >= rate_of_return]
    
    ## Construct and solve the problem
    prob = cvx.Problem(objective, constraints)
    prob.solve()
    
    return reduced_weights.value, etfs_selection.value

# Determine optimal weights of ETFs

In [170]:
annual_expected_return = 0.1
annual_expense_ratio = 0.0075
first_year_returns = etf_returns_df[:252]
covariance_returns = get_covariance_returns(first_year_return)
asset_weights = None
rate_of_return = annual_expected_return
while asset_weights is None:
    asset_weights, assets_selection = get_optimal_weights(covariance_returns, first_year_returns.values, 
                                                          rate_of_return + annual_expense_ratio)
    rate_of_return -= 0.01
    
asset_weights = [round(weight, 4) for weight in asset_weights]
assets_tickers = list(first_year_return.columns)
optimal_asset_weights = {}
for i in range(len(assets_tickers)):
    if abs(assets_selection[i] - 1) < 0.001 and asset_weights[i] > 0.01:
        optimal_asset_weights[assets_tickers[i]] = asset_weights[i]
        
optimal_asset_weights['date'] = etf_returns_df.index[252]
optimal_asset_weights = adjust_weights(optimal_asset_weights)
optimal_asset_weights

{'AGG': 0.232,
 'EWZ': 0.1177,
 'IAU': 0.0711,
 'SDS': 0.059,
 'SH': 0.3,
 'SPY': 0.0764,
 'USO': 0.03939999999999989,
 'XLU': 0.1044,
 'date': '2008-01-02'}

# Rebalance Portfolio Over Time

Adopt a calendar-based rebalancing every 3 months

In [17]:
# def rebalance_portfolio(returns, expected_return, expense_ratio,
#                         shift_size, chunk_size):
#     """
#     Get weights for each rebalancing of the portfolio.

#     Parameters
#     ----------
#     returns : DataFrame
#         Returns for each ticker and date
#     expected_return : float
#         Expected rate of return for the portfolio
#     shift_size : int
#         The number of days between each rebalance
#     chunk_size : int
#         The number of days to look in the past for rebalancing

#     Returns
#     -------
#     all_rebalance_weights  : list of Ndarrays
#         The ETF weights for each point they are rebalanced
#     """
#     #yearly_trading_days = 252
#     all_rebalance_weights = []
#     dates_with_rebalance = []
#     assets_tickers = list(returns.columns)
#     num_assets = len(assets_tickers)
#     for end in np.arange(shift_size, len(returns), shift_size):
#         rate_of_return = expected_return
#         if end - chunk_size < 0:
#             continue
#         start = end - chunk_size
#         partial_returns = returns[start:end]
#         #dates_with_rebalance.append(returns.index[end])
#         covariance_returns = get_covariance_returns(partial_returns)
#         rebalanced_asset_weights = None
#         while rebalanced_asset_weights is None:
#             rebalanced_asset_weights, assets_selection = get_optimal_weights(covariance_returns,
#                                                                              partial_returns.values,
#                                                                              rate_of_return + expense_ratio)
#             rate_of_return -= 0.01
#         rebalanced_asset_weights = [round(weight, 4) for weight in rebalanced_asset_weights]
#         selected_asset_weights = {}
#         for i in range(num_assets):
#             if abs(assets_selection[i] - 1) < 0.001 and rebalanced_asset_weights[i] > 0.01:
#                 selected_asset_weights[assets_tickers[i]] = rebalanced_asset_weights[i]
#         print("Rebalanced portfolios on %s:" % returns.index[end])
#         print(selected_asset_weights)
#         print("Actual achievable return: %s" % round(rate_of_return + 0.01, 4))
#         print("===========================================")
#         print()
#         selected_asset_weights['date'] = returns.index[end]
#         all_rebalance_weights.append(selected_asset_weights)

#     return all_rebalance_weights

In [35]:
# # Acquire ETF price data
# etf_prices = pd.read_csv('data/prices.txt', sep=',')
# etf_prices = etf_prices.rename(columns={etf_prices.columns[0]: "date"}).set_index('date')

# # Remove US dollar
# etf_prices = etf_prices.iloc[:, :-1]
# #etf_prices = etf_prices.sample(n=20, random_state=1, axis=1)
# #etf_prices = etf_prices[:600]
# print("Number of assets: %s" %etf_prices.shape[1])
# print("Number of samples: %s" %etf_prices.shape[0])

# # Calculate log return
# etf_returns_df = np.log(etf_prices) - np.log(etf_prices.shift(1))
# etf_returns_df = etf_returns_df.dropna()

# chunk_size = 252
# shift_size = 21
# expected_return = 0.2
# annual_expense_ratio = 0.0075
# print("Rebalancing portfolios...")
# start_time = time.time()
# all_rebalanced_weights = rebalance_portfolio(etf_returns_df, expected_return,
#                                              annual_expense_ratio, shift_size, chunk_size)
# print("Elapsed time: %s seconds" %(round(time.time() - start_time, 4)))
# adjusted_rebalanced_weights = []

# # Reduce ETFs with negligible weights
# for weight in all_rebalanced_weights:
#     adjusted_rebalanced_weights.append(adjust_weights(weight))
    
# with open('portfolio.json', 'w') as f:
#     json.dump(adjusted_rebalanced_weights, f, indent=4)

Number of assets: 52
Number of samples: 2521
Rebalancing portfolios...
Rebalanced portfolios on 2008-01-02:
{'AGG': 0.2217, 'DBC': 0.1227, 'EWZ': 0.241, 'IAU': 0.1424, 'SDS': 0.2682}
Actual achievable return: 0.2

Rebalanced portfolios on 2008-02-01:
{'AGG': 0.293, 'EWZ': 0.1365, 'IAU': 0.188, 'OIL': 0.0786, 'SH': 0.3}
Actual achievable return: 0.2

Rebalanced portfolios on 2008-03-04:
{'AGG': 0.0116, 'DIA': 0.2564, 'EWZ': 0.1821, 'SDS': 0.1578, 'SH': 0.3, 'USO': 0.0835}
Actual achievable return: 0.2

Rebalanced portfolios on 2008-04-03:
{'DBC': 0.223, 'DIA': 0.2887, 'EWZ': 0.1622, 'SDS': 0.2973, 'USO': 0.0259}
Actual achievable return: 0.2

Rebalanced portfolios on 2008-05-02:
{'AGG': 0.3, 'DIA': 0.0916, 'EWZ': 0.0948, 'FXI': 0.0379, 'OIL': 0.1717, 'SH': 0.3}
Actual achievable return: 0.2

Rebalanced portfolios on 2008-06-03:
{'AGG': 0.3, 'DBC': 0.0556, 'EWZ': 0.0612, 'FXI': 0.0388, 'IWM': 0.1036, 'OIL': 0.1398, 'SH': 0.3}
Actual achievable return: 0.2

Rebalanced portfolios on 2008-0

Rebalanced portfolios on 2011-08-10:
{'AGG': 0.293, 'GLD': 0.1768, 'IAU': 0.3, 'SH': 0.0464, 'XLK': 0.1808}
Actual achievable return: 0.2

Rebalanced portfolios on 2011-09-09:
{'AGG': 0.3, 'GLD': 0.1009, 'IAU': 0.3, 'IBB': 0.1516, 'SH': 0.0606, 'SPY': 0.0829}
Actual achievable return: 0.2

Rebalanced portfolios on 2011-10-10:
{'GLD': 0.3, 'IAU': 0.3, 'IBB': 0.1821, 'SLV': 0.2139}
Actual achievable return: 0.2

Rebalanced portfolios on 2011-11-08:
{'GLD': 0.3, 'IAU': 0.3, 'XLP': 0.2582, 'XOP': 0.1368}
Actual achievable return: 0.2

Rebalanced portfolios on 2011-12-08:
{'GLD': 0.3, 'IAU': 0.3, 'IBB': 0.1323, 'XLP': 0.2647}
Actual achievable return: 0.19

Rebalanced portfolios on 2012-01-10:
{'DIA': 0.1044, 'GLD': 0.3005, 'IAU': 0.3005, 'IBB': 0.2991}
Actual achievable return: 0.15

Rebalanced portfolios on 2012-02-09:
{'AGG': 0.0255, 'GLD': 0.0852, 'IAU': 0.3, 'IBB': 0.2853, 'LQD': 0.3}
Actual achievable return: 0.2

Rebalanced portfolios on 2012-03-12:
{'GLD': 0.3, 'IAU': 0.2501, 'IBB':

Rebalanced portfolios on 2015-06-10:
{'AGG': 0.0283, 'DXJ': 0.3, 'IBB': 0.3, 'SPY': 0.0797, 'VTI': 0.287}
Actual achievable return: 0.2

Rebalanced portfolios on 2015-07-10:
{'AGG': 0.1352, 'IBB': 0.3, 'ITB': 0.2501, 'SH': 0.0118, 'XLY': 0.3}
Actual achievable return: 0.19

Rebalanced portfolios on 2015-08-10:
{'IBB': 0.3, 'ITB': 0.1758, 'SH': 0.2222, 'XLV': 0.3}
Actual achievable return: 0.2

Rebalanced portfolios on 2015-09-09:
{'IBB': 0.3004, 'ITB': 0.299, 'XLF': 0.1006, 'XLV': 0.2987}
Actual achievable return: 0.18

Rebalanced portfolios on 2015-10-08:
{'DFE': 0.2999, 'ITB': 0.3, 'KRE': 0.2998, 'SPY': 0.1016}
Actual achievable return: 0.12

Rebalanced portfolios on 2015-11-06:
{'IBB': 0.2995, 'ITB': 0.2997, 'SPY': 0.1007, 'XLY': 0.3}
Actual achievable return: 0.13

Rebalanced portfolios on 2015-12-08:
{'DFE': 0.2964, 'ITB': 0.2948, 'SPY': 0.1146, 'XHB': 0.2952}
Actual achievable return: 0.09

Rebalanced portfolios on 2016-01-08:
{'DFE': 0.2996, 'LQD': 0.1079, 'SH': 0.2997, 'XLY': 0

In [36]:
adjusted_rebalanced_weights_df = pd.DataFrame(data=adjusted_rebalanced_weights)
#adjusted_rebalanced_weights_df = adjusted_rebalanced_weights_df.set_index('date')
adjusted_rebalanced_weights_df = adjusted_rebalanced_weights_df.fillna(0)
adjusted_rebalanced_weights_df

,AGG,DBC,DFE,DIA,DXJ,EWH,EWT,EWW,EWY,EWZ,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP,date
0,0.2217,0.1267,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.2410,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-01-02
1,0.2930,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.1365,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-02-01
2,0.0202,0.0000,0.0000,0.2564,0.0000,0.0,0.0000,0.0000,0.000,0.1821,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-03-04
3,0.0000,0.2230,0.0000,0.2887,0.0000,0.0,0.0000,0.0000,0.000,0.1622,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-04-03
4,0.3000,0.0000,0.0000,0.0916,0.0000,0.0,0.0000,0.0000,0.000,0.0948,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-05-02
5,0.3000,0.0556,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.0612,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-06-03
6,0.3000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.0591,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0678,2008-07-02
7,0.0000,0.1886,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.0943,...,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-08-01
8,0.0000,0.1156,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.0651,...,0.0000,0.0,0.0000,0.3000,0.0000,0.0000,0.0000,0.0000,0.0000,2008-09-02
9,0.0000,0.0000,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.000,0.0000,...,0.0000,0.0,0.0000,0.1067,0.0000,0.0000,0.0000,0.0000,0.0000,2008-10-01


# Maintain optimal asset weights over time

In [182]:
optimal_portfolio_etfs = list(optimal_asset_weights.keys())
optimal_portfolio_etfs.remove('date')
optimal_portfolio_holding_shares_df = pd.DataFrame(columns=optimal_portfolio_etfs)
optimal_portfolio_holding_shares_df['date'] = adjusted_rebalanced_weights_df['date']
optimal_portfolio_holding_shares_df['balance'] = 0
optimal_portfolio_holding_shares_df = optimal_portfolio_holding_shares_df.fillna(0)
#optimal_portfolio_holding_shares_df = optimal_portfolio_holding_shares_df.set_index('date')
optimal_portfolio_holding_shares_df.head()

,AGG,EWZ,IAU,SDS,SH,SPY,USO,XLU,date,balance
0,0,0,0,0,0,0,0,0,2008-01-02,0
1,0,0,0,0,0,0,0,0,2008-04-03,0
2,0,0,0,0,0,0,0,0,2008-07-02,0
3,0,0,0,0,0,0,0,0,2008-10-01,0
4,0,0,0,0,0,0,0,0,2008-12-29,0


In [19]:
initial_balance = 10000
rebalance_count = 0
while rebalance_count < optimal_portfolio_holding_shares_df.shape[0]:
    if rebalance_count == 0:
        balance = initial_balance
    else:
        balance = 0
        for etf in optimal_portfolio_etfs:
            balance += optimal_portfolio_holding_shares_df.loc[rebalance_count-1, etf] * partial_etf_prices.loc[rebalance_count, etf]
    optimal_portfolio_holding_shares_df.loc[rebalance_count, 'balance'] = round(balance, 2)
    for etf in optimal_portfolio_etfs:
        optimal_portfolio_holding_shares_df.loc[rebalance_count, etf] = balance * optimal_asset_weights.get(etf) / partial_etf_prices.loc[rebalance_count, etf]
        optimal_portfolio_holding_shares_df.loc[rebalance_count, etf] = round(optimal_portfolio_holding_shares_df.loc[rebalance_count, etf], 2)
    rebalance_count += 1

optimal_portfolio_holding_shares_df = optimal_portfolio_holding_shares_df.set_index('date')
optimal_portfolio_holding_shares_df

# Calculate cumulative returns over time

In [37]:
holding_shares_df = pd.DataFrame(columns=adjusted_rebalanced_weights_df.columns)
holding_shares_df['date'] = adjusted_rebalanced_weights_df['date']
holding_shares_df['balance'] = 0
holding_shares_df = holding_shares_df.fillna(0)
holding_shares_df.head()

,AGG,DBC,DFE,DIA,DXJ,EWH,EWT,EWW,EWY,EWZ,...,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP,date,balance
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008-01-02,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008-02-01,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008-03-04,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008-04-03,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2008-05-02,0


### Calculate number of holding shares for each asset
Assume constant amount of $10,000 maintained in the portfolio across the period of time. The number of shares of each asset can be calculated.

In [38]:
dates_with_rebalance = adjusted_rebalanced_weights_df['date']
cols =  list(adjusted_rebalanced_weights_df.columns)
cols.remove('date')
partial_etf_prices = etf_prices.loc[dates_with_rebalance, cols].reset_index()
partial_etf_prices.head()

,date,AGG,DBC,DFE,DIA,DXJ,EWH,EWT,EWW,EWY,...,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP
0,2008-01-02,101.65,32.43,59.88,130.63,50.60,21.79,14.49,54.62,62.57,...,79.50,28.36,38.51,26.12,28.39,42.09,34.94,32.20,69.37,52.90
1,2008-02-01,102.99,31.91,56.26,127.43,49.67,20.10,14.25,56.80,57.55,...,71.35,29.68,37.85,23.65,27.47,40.24,33.86,32.87,68.36,49.09
2,2008-03-04,102.09,36.50,56.93,122.40,47.64,18.21,15.55,56.15,56.50,...,75.50,25.34,36.35,22.17,27.38,38.84,32.83,31.20,73.84,53.99
3,2008-04-03,101.60,36.49,59.36,126.25,49.64,18.93,16.21,61.73,59.17,...,76.57,26.72,38.25,23.32,28.13,39.35,31.52,32.20,74.12,55.35
4,2008-05-02,101.93,37.97,58.35,130.73,50.97,20.04,16.90,59.30,61.35,...,81.47,27.67,38.90,24.79,28.11,41.02,32.05,33.19,77.60,58.87


In [39]:
holding_shares_df = pd.DataFrame(columns=adjusted_rebalanced_weights_df.columns)
holding_shares_df['date'] = adjusted_rebalanced_weights_df['date']
holding_shares_df['balance'] = 0
holding_shares_df = holding_shares_df.fillna(0)
etfs = list(adjusted_rebalanced_weights_df.columns)
etfs.remove('date')
initial_balance = 10000
rebalance_count = 0
while rebalance_count < holding_shares_df.shape[0]:
    if rebalance_count == 0:
        balance = initial_balance
    else:
        balance = 0
        for etf in etfs:
            balance += holding_shares_df.loc[rebalance_count-1, etf] * partial_etf_prices.loc[rebalance_count, etf]
    holding_shares_df.loc[rebalance_count, 'balance'] = round(balance, 2)
    for etf in etfs:
        holding_shares_df.loc[rebalance_count, etf] = balance * adjusted_rebalanced_weights_df.loc[rebalance_count, etf] / partial_etf_prices.loc[rebalance_count, etf]
        holding_shares_df.loc[rebalance_count, etf] = round(holding_shares_df.loc[rebalance_count, etf], 2)
    rebalance_count += 1

holding_shares_df = holding_shares_df.set_index('date')
holding_shares_df

,AGG,DBC,DFE,DIA,DXJ,EWH,EWT,EWW,EWY,EWZ,...,XLF,XLI,XLK,XLP,XLU,XLV,XLY,XME,XOP,balance
date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,21.81,39.07,0.00,0.00,0.00,0.0,0.00,0.00,0.00,29.77,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10000.00
2008-02-01,28.98,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,17.84,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10185.96
2008-03-04,2.10,0.00,0.00,22.25,0.00,0.0,0.00,0.00,0.00,23.08,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10624.05
2008-04-03,0.00,63.93,0.00,23.92,0.00,0.0,0.00,0.00,0.00,20.51,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10460.95
2008-05-02,31.54,0.00,0.00,7.51,0.00,0.0,0.00,0.00,0.00,10.88,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10716.81
2008-06-03,32.14,14.97,0.00,0.00,0.00,0.0,0.00,0.00,0.00,6.94,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10837.14
2008-07-02,33.59,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,7.85,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,10.98,11147.36
2008-08-01,0.00,50.97,0.00,0.00,0.00,0.0,0.00,0.00,0.00,12.89,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,10812.25
2008-09-02,0.00,32.54,0.00,0.00,0.00,0.0,0.00,0.00,0.00,9.61,...,0.00,0.00,0.00,109.16,0.00,0.00,0.00,0.0,0.00,10351.99


# Generate random portfolios

10 random portfolios with 5-10 ETFs are simulated.

In [40]:
def generate_rand_weights(num_assets):
    seeds = []
    for i in range(num_assets):
        seeds.append(random.randint(1,10))
    
    return [round(seed / sum(seeds), 4) for seed in seeds]

In [41]:
N = 10 # No of random portfolios
portfolios = pd.DataFrame()
portfolios['Optimized Portfolio'] = holding_shares_df['balance']

for portfolio_id in range(N):
    # Generate no of assets in the portfolio
    n_etfs = random.randint(5,10)
    etf_pools = list(etf_prices.columns)
    rand_etfs = dict(zip(random.sample(etf_pools, n_etfs), 
                         generate_rand_weights(n_etfs)))
    
    # Initialize random portfolio
    porfolio_name = 'Random Portfolio ' + str(portfolio_id+1)
    portfolios[porfolio_name] = 0
    initial_balance = 10000
    holding_shares = {}
    start_point = '2008-01-02'
    for etf in rand_etfs:
        holding_shares[etf] = initial_balance * rand_etfs.get(etf) / etf_prices.loc[start_point, etf]

    for compare_point in portfolios.index:
        if compare_point == start_point:
            balance = initial_balance
        else:
            balance = 0
            for etf in rand_etfs:
                balance += holding_shares.get(etf) * etf_prices.loc[compare_point, etf]
                
        balance = round(balance, 2)
        portfolios.loc[compare_point, porfolio_name] = balance

portfolios.head()

,Optimized Portfolio,Random Portfolio 1,Random Portfolio 2,Random Portfolio 3,Random Portfolio 4,Random Portfolio 5,Random Portfolio 6,Random Portfolio 7,Random Portfolio 8,Random Portfolio 9,Random Portfolio 10
date,,,,,,,,,,,
2008-01-02,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00
2008-02-01,10185.96,9601.75,9872.16,9912.89,9908.07,9620.10,9561.25,9818.47,9614.79,9629.87,10008.05
2008-03-04,10624.05,9380.57,9505.68,9672.95,9965.46,9283.47,9515.74,9754.39,9332.55,9931.11,9599.92
2008-04-03,10460.95,9740.31,9991.48,9817.41,10196.70,9499.64,9727.84,9789.41,9609.44,9845.72,10263.30
2008-05-02,10716.81,10088.92,10120.86,9900.56,10364.56,9891.76,10055.14,9987.69,9727.79,10072.24,10495.17


### Plot portfolio holding value over time

In [42]:
portfolio_holdings = []
portfolios['Optimized Portfolio'] = optimal_portfolio_holding_shares_df['balance']
for portfolio in portfolios.columns:
    if portfolio == 'Optimized Portfolio':
        trace = go.Scatter(
            x = portfolios.index,
            y = portfolios[portfolio],
            name = portfolio,
            line = dict(color = '#FC4A1A'),
            opacity = 0.8
        )
    else:
        trace = go.Scatter(
            x = portfolios.index,
            y = portfolios[portfolio],
            name = portfolio,
            line = dict(color = '#7F7F7F'),
            opacity = 0.8
        )

    portfolio_holdings.append(trace)

layout = dict(
    title = "Portfolio Holding Values from 2008 to 2016",
    xaxis = dict(
        range = [portfolios.index])
)

fig = dict(data=portfolio_holdings, layout=layout)
py.iplot(fig, filename = "Portfolio Holding Values from 2008 to 2016")

In [6]:
import math
math.exp(0.1)

1.1051709180756477